In [1]:
import numpy as np

# Step 1: Define Input Parameters
# -------------------------------

In [2]:
# Camera parameters
f = 18  # Focal length in mm
pixel_size = 0.005  # Pixel size in mm
sigma_pixels = 3  # Standard deviation of image measurements in pixels
sigma_x = sigma_y = sigma_pixels * pixel_size  # Standard deviation in mm
print(f"Standard deviation of image measurements: {sigma_x:.3f} mm")

Standard deviation of image measurements: 0.015 mm


In [3]:
# Exterior orientation parameters (in meters and degrees)
# Left camera
X_L, Y_L, Z_L = -7.5, -10.0, 1.7  # Position in meters
w_L, phi_L, k_L = np.radians(90), np.radians(-20), np.radians(0)  # Angles in radians

# Right camera
X_R, Y_R, Z_R = 7.5, -10.0, 1.7  # Position in meters
w_R, phi_R, k_R = np.radians(90), np.radians(20), np.radians(0)  # Angles in radians

# Coordinates of intersection point P (in meters)
X0, Y0, Z0 = 0.05, 0.0, 3.1


# Step 2: Define Rotation Matrix Function
# --------------------------------------

In [4]:
def rotation_matrix(w, phi, k):
    """
    Compute the rotation matrix R = R_kappa * R_phi * R_omega.
    
    Parameters:
    w (float): Omega angle in radians (rotation around X-axis).
    phi (float): Phi angle in radians (rotation around Y-axis).
    k (float): Kappa angle in radians (rotation around Z-axis).
    
    Returns:
    numpy.ndarray: 3x3 rotation matrix.
    """
    R_w = np.array([[1, 0, 0],
                    [0, np.cos(w), -np.sin(w)],
                    [0, np.sin(w), np.cos(w)]])
    R_phi = np.array([[np.cos(phi), 0, np.sin(phi)],
                      [0, 1, 0],
                      [-np.sin(phi), 0, np.cos(phi)]])
    R_k = np.array([[np.cos(k), -np.sin(k), 0],
                    [np.sin(k), np.cos(k), 0],
                    [0, 0, 1]])
    return R_k @ R_phi @ R_w

# Compute rotation matrices for both cameras
R_L = rotation_matrix(w_L, phi_L, k_L)
R_R = rotation_matrix(w_R, phi_R, k_R)


# Step 3: Define Design Matrix Function
# ------------------------------------

In [5]:
def design_matrix_row(X, Y, Z, R, Xc, Yc, Zc, f):
    """
    Compute the partial derivatives of image coordinates (x, y) with respect to
    object coordinates (X, Y, Z) for one camera, based on collinearity equations.
    
    Parameters:
    X, Y, Z (float): Object coordinates in meters.
    R (numpy.ndarray): 3x3 rotation matrix.
    Xc, Yc, Zc (float): Camera position in meters.
    f (float): Focal length in mm.
    
    Returns:
    numpy.ndarray: 2x3 matrix of partial derivatives [dx/dX, dx/dY, dx/dZ; dy/dX, dy/dY, dy/dZ].
    """
    # Vector from camera to point
    dX = X - Xc
    dY = Y - Yc
    dZ = Z - Zc
    
    # Rotate the vector
    RX = R @ np.array([dX, dY, dZ])
    
    # Rotation matrix elements
    r11, r12, r13 = R[0]
    r21, r22, r23 = R[1]
    r31, r32, r33 = R[2]
    
    # Denominator of collinearity equations
    q = r31 * dX + r32 * dY + r33 * dZ
    dq_dX = r31
    dq_dY = r32
    dq_dZ = r33
    
    # Numerators
    num_x = r11 * dX + r12 * dY + r13 * dZ
    num_y = r21 * dX + r22 * dY + r23 * dZ
    
    # Partial derivatives
    dx_dX = -f * (r11 * q - num_x * dq_dX) / q**2
    dx_dY = -f * (r12 * q - num_x * dq_dY) / q**2
    dx_dZ = -f * (r13 * q - num_x * dq_dZ) / q**2
    
    dy_dX = -f * (r21 * q - num_y * dq_dX) / q**2
    dy_dY = -f * (r22 * q - num_y * dq_dY) / q**2
    dy_dZ = -f * (r23 * q - num_y * dq_dZ) / q**2
    
    return np.array([[dx_dX, dx_dY, dx_dZ],
                     [dy_dX, dy_dY, dy_dZ]])

# Step 4: Compute Design Matrix
# -----------------------------

In [6]:
# Design matrix for left and right images
A_L = design_matrix_row(X0, Y0, Z0, R_L, X_L, Y_L, Z_L, f)
A_R = design_matrix_row(X0, Y0, Z0, R_R, X_R, Y_R, Z_R, f)

# Combine into full design matrix (4 observations x 3 unknowns)
A = np.vstack([A_L, A_R])
print("\nDesign matrix A:")
print(A)

# Step 5: Error Propagation
# -------------------------
# Covariance matrix of observations (assuming equal variance and no correlation)
sigma_sq = sigma_x**2  # Variance in mm^2
Sigma_ll_inv = np.eye(4) / sigma_sq  # Inverse covariance matrix

# Normal matrix
N = A.T @ Sigma_ll_inv @ A

# Covariance matrix of estimated parameters
Q_xx = np.linalg.inv(N)

# Standard deviations of X, Y, Z
sigma_XYZ = np.sqrt(np.diag(Q_xx))

# Step 6: Display Results
# -----------------------
print("\nStandard deviations of point P coordinates (in meters):")
print(f"sigma_X = {sigma_XYZ[0]:.4f} m")
print(f"sigma_Y = {sigma_XYZ[1]:.4f} m")
print(f"sigma_Z = {sigma_XYZ[2]:.4f} m")


Design matrix A:
[[-1.25434917e+00  9.47033625e-01  5.79890849e-17]
 [-6.00617757e-02 -1.65018373e-01  1.50260724e+00]
 [-1.26154260e+00 -9.39849241e-01 -5.75491682e-17]
 [ 6.04062175e-02 -1.65964719e-01  1.50690965e+00]]

Standard deviations of point P coordinates (in meters):
sigma_X = 0.0084 m
sigma_Y = 0.0112 m
sigma_Z = 0.0072 m
